# ラベルのないデータから機械学習のモデルを構築する: 画像分類における SageMaker Ground Truth のデモ
<!-- From Unlabeled Data to a Deployed Machine Learning Model: A SageMaker Ground Truth Demonstration for Image Classification
 -->
1. [イントロダクション](#イントロダクション)
2. [Ground Truth ラベリングジョブの実行 (目安の時間: 3 時間)](#Ground-Truth-ラベリングジョブの実行)
    1. [データの準備](#データの準備)
    2. [カテゴリを特定する](#カテゴリを特定する)
    3. [指示書テンプレートの作成](#指示書テンプレートの作成)
    4. [タスクをテストするためのプライベートチームを作成する [オプション]](#タスクをテストするためのプライベートチームを作成する-[オプション])
    5. [ラベリングジョブで使用する構築済み Lambda 関数を定義する](#ラベリングジョブで使用する構築済み-Lambda-関数を定義する)
    6. [Ground Truth ジョブリクエストを送信する](#Ground-Truth-ジョブリクエストを送信する)
        1. [Vプライベートチームでタスクを検証する [オプション]](#プライベートチームでタスクを検証する-[オプション])
    7. [ジョブの進捗状況を監視する](#ジョブの進捗状況を監視する)
3. [Ground Truth ラベリングジョブの結果を分析する (目安の時間: 20 分)](#Ground-Truth-ラベリングジョブの結果を分析する)
    1. [出力マニフェストファイルの結果を後処理する](#出力マニフェストファイルの結果を後処理する)
    2. [クラスに対するヒストグラムを作成する](#クラスに対するヒストグラムを作成する)
    3. [アノテーションされた画像を表示する](#アノテーションされた画像を表示する)
        1. [数枚の出力サンプルを表示する](#数枚の出力サンプルを表示する)
        2. [すべての結果を表示する](#すべての結果を表示する)
4. [Ground Truth の結果を、既知の事前にラベル付けされたデータと比較する (目安の時間: 5 分)](#Ground-Truth-の結果を、既知の事前にラベル付けされたデータと比較する)
    1. [精度を計算する](#精度を計算する)
    2. [正誤のアノテーションをプロットする](#正誤のアノテーションをプロットする)
5. [Ground Truth を使った画像分類器のトレーニング (目安の時間: 15 分)](#Ground-Truth-を使った画像分類器のトレーニング)
6. [モデルをデプロイする (目安の時間: 20 分)](#モデルをデプロイする)
    1. [モデルの生成](#モデルの生成)
    2. [バッチ変換](#バッチ変換)
    3. [リアルタイム推論](#リアルタイム推論)
        1. [エンドポイント設定を作成する](#エンドポイント設定を作成する)
        2. [エンドポイントを作成する](#エンドポイントを作成する)
        3. [推論を実行する](#推論を実行する)
7. [まとめ](#まとめ)

## イントロダクション
このサンプルノートブックでは、SageMaker Ground Truth の機能を end-to-end のワークフローを通して体験することができます。まずラベルがついていない画像のデータセットからはじめ、SageMaker Ground Truth を使ってラベルを取得し、ラベリングジョブの結果を解析し、画像分類機の学習を行い、できたモデルをホストし、そして最後にそれを使った推論を行います。始める前に、ご自身がワークフローに慣れるために、まずは AWS コンソールで Ground Truth のラベリングジョブを始めることを強く推奨します。AWS コンソールは API に比べて柔軟性が低いものになりますが、使用は簡単です。

#### 費用とランタイム
このデモは２つのモードで実行できます：
1. 次のセルで`RUN_FULL_AL_DEMO = True`とすることで、1000枚の画像のラベル付けを行います。費用は現在の [Ground Truth の料金体系](https://aws.amazon.com/jp/sagemaker/data-labeling/pricing/)だと、およそ \$80 かかります。費用を削減するために、このデモでは、Ground Truth のラベル付けの自動化の機能を使用します。この機能はコンピュータビジョンにより人間による回答を学習し、安い料金で最も簡単な画像のラベルを自動で生成します。すべてのランタイムを実行するにはおよそ 4 時間かかります。
1. 次のセルで`RUN_FULL_AL_DEMO = False`とすることで、100枚だけの画像のラベル付けを行います。費用は \$8 ほどです。**Ground Truth のラベル付けの自動化の機能は、1000枚以上の画像のデータセットのみに対して実行されるため、デモのこの安価な版では使用しません。下記で解析した結果の図はもしかしたら違和感があるように見えるかもしれませんが、人間による100枚の画像でも良い結果を見ることができます。**

#### 前提条件
各セルを１つずつ実行するだけで、このノートブックは実行できます。実際に何をしているかを理解するためには、下記のことが必要になります：
* 書き込み可能な S3 バケット -- 次のセルにバケット名を記載してください。バケットは SageMaker ノートブックインスタンスと同じリージョンにいなければなりません。また、`EXP_NAME` は任意の S3 プリフィクスに変更できます。この試行に関連するすべてのファイルは指定されたバケットのプリフィクスに格納されます。
* このデモで使う S3 バケットは、CORS ポリシーが付与されている必要があります。この条件やどのように CORS ポリシーを S3 バケットに付与するかについては、[CORS Permission Requirement](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-cors-update.html) をご確認ください。
* Python と [numpy](http://www.numpy.org/) の知識
* [AWS S3](https://docs.aws.amazon.com/s3/index.html) の基礎的な知識
* [AWS Sagemaker](https://aws.amazon.com/sagemaker/) の基礎的な理解
* [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/) の基礎的な知識 -- このノートブックを実行している AWS アカウントへのアクセスのための認証情報をセットアップしてください。それにより SageMaker Jupyter ノートブックインスタンスの枠を超えて実行することができます。

このノートブックは SageMaker ノートブックインスタンスでのみテストされています。指定されているランタイムはおおよそのものであり、テストでは `ml.m4.xlarge` インスタンスを使っています。しかし、初めに次のセルを SageMaker で実行し、ローカルにコピーしたノートブックに `role` の文字列をコピーすることで、ローカルのインスタンスでも実行可能です。

注：このノートブックは作業用のディレクトリにサブディレクトリを作成・削除します。実行前に、このノートブックを自身のディレクトリに配置することをお勧めします。

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import namedtuple
from collections import defaultdict
from collections import Counter
import itertools
import json
import random
import time
import imageio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import confusion_matrix
import boto3
import sagemaker
from urllib.parse import urlparse


BUCKET = "<< YOUR S3 BUCKET NAME >>"
assert BUCKET != "<< YOUR S3 BUCKET NAME >>", "Please provide a custom S3 bucket name."
EXP_NAME = "ground-truth-ic-demo"  # 有効な任意の S3 prefix
RUN_FULL_AL_DEMO = False  # 上述の '費用とランタイム' を確認してください!

In [ ]:
# ノートブックが S3 バケットと同じリージョンにいることを確認してください。
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name
s3 = boto3.client("s3")
bucket_region = s3.head_bucket(Bucket=BUCKET)["ResponseMetadata"]["HTTPHeaders"][
    "x-amz-bucket-region"
]
assert (
    bucket_region == region
), "You S3 bucket {} and this notebook need to be in the same region.".format(BUCKET)

# Ground Truth ラベリングジョブの実行
**このセクションは完了に3時間程度かかります。**

まずはじめにラベリングジョブを実行します。このためには複数のステップを実行します。ラベル付けを行いたいデータを収集し、可能性のあるカテゴリを特定し、指示書を作成し、ラベリングジョブの仕様を記述します。加えて、パブリックワークフォースにジョブを公開する前に、プライベートワークフォースを利用して、（無料の）擬似のジョブを実行することを強く推奨します。このノートブックではオプションとして、その手順を説明します。プライベートワークフォースなしでも、ラベリングジョブの完了までに３時間ほどかかります。しかし、これはパブリックのアノテーションワークフォースの利用可能状況によって変わります。

## データの準備
まず [Google Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html) から画像とラベルの一部をダウンロードします。これらのラベルは[正当性が慎重に保証されています](https://storage.googleapis.com/openimages/web/factsfigures.html)。あとの方で、このラベルと Ground Truth のアノテーションの結果を比較します。今回のデータセットの画像は次のカテゴリーを含みます：

* 音楽楽器（Musical Instrument、500枚)
* フルーツ（Fruit、370枚)
* チーター（Cheetah、50枚)
* 虎（Tiger、40枚)
* 雪だるま（Snowman、40枚)

`RUN_FULL_AL_DEMO = False` を選択した場合、このデータセットのうち100枚の画像が使われます。このデータセットは、面白みのある画像を含んでいるので、アノテーターにとっても楽しいものになるはずです。アノテーターにはどんな画像でも望むもののラベル付けを依頼することができます（ただし、アダルトコンテンツを含まない場合に限ります。この場合、ラベリングジョブのリクエストをそのジョブ用の方法に調整する必要があります。詳しくは Ground Truth の開発者ガイドをご覧ください）。

これらの画像は、ローカルの `BUCKET` にコピーされ、対応する*入力マニフェスト*を生成します。この入力マニフェストは、Ground Truth にアノテーションを行わせたい画像の S3 内の位置の整形化リストです。これを S3 `BUCKET` にアップロードします。

#### Open Images Dataset V4 に関する公開：
Open Images Dataset V4 は Google Inc.　によって作成されています. 画像や付随しているアノテーションを修正していません。画像とアノテーションについては、 [こちら](https://storage.googleapis.com/openimages/web/download.html)から取得できます。アノテーションは [CC BY 4.0](https://creativecommons.org/licenses/by/2.0/) ライセンスのもと、Google Inc. によってライセンスを提供されています。 画像は [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/) ライセンスを持つとしてリストに載っています。次の論文では. Open Images V4 について、データ収集やアノテーションからデータの統計情報やそれに基づいて学習されたモデルの評価まで、深く説明しています。

A. Kuznetsova, H. Rom, N. Alldrin, J. Uijlings, I. Krasin, J. Pont-Tuset, S. Kamali, S. Popov, M. Malloci, T. Duerig, and V. Ferrari.
*The Open Images Dataset V4: Unified image classification, object detection, and visual relationship detection at scale.* arXiv:1811.00982, 2018. ([PDFのリンク](https://arxiv.org/abs/1811.00982))

In [ ]:
# Open Images のアノテーションをダウンロードし、処理する。
!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-human-imagelabels-boxable.csv -O openimgs-annotations.csv
with open("openimgs-annotations.csv", "r") as f:
    all_labels = [line.strip().split(",") for line in f.readlines()]

# 各カテゴリのイメージ ID を取得
ims = {}
ims["Musical Instrument"] = [
    label[0] for label in all_labels if (label[2] == "/m/04szw" and label[3] == "1")
][:500]
ims["Fruit"] = [label[0] for label in all_labels if (label[2] == "/m/02xwb" and label[3] == "1")][
    :371
]
ims["Fruit"].remove(
    "02a54f6864478101"
)  # この画像は個人情報を含むため、データセットから削除します。
ims["Cheetah"] = [label[0] for label in all_labels if (label[2] == "/m/0cd4d" and label[3] == "1")][
    :50
]
ims["Tiger"] = [label[0] for label in all_labels if (label[2] == "/m/07dm6" and label[3] == "1")][
    :40
]
ims["Snowman"] = [
    label[0] for label in all_labels if (label[2] == "/m/0152hh" and label[3] == "1")
][:40]
num_classes = len(ims)

# 短縮版のデモの場合、各クラスの最初の1/10を使用する。
for key in ims.keys():
    if RUN_FULL_AL_DEMO is False:
        ims[key] = set(ims[key][: int(len(ims[key]) / 10)])
    else:
        ims[key] = set(ims[key])

In [ ]:
# ローカルのバケットに画像をコピーする。
s3 = boto3.client("s3")
for img_id, img in enumerate(itertools.chain.from_iterable(ims.values())):
    if (img_id + 1) % 10 == 0:
        print("Copying image {} / {}".format((img_id + 1), 1000))
    copy_source = {"Bucket": "open-images-dataset", "Key": "test/{}.jpg".format(img)}
    s3.copy(copy_source, BUCKET, "{}/images/{}.jpg".format(EXP_NAME, img))

# インプットマニフェストを作成してアップロードする。
manifest_name = "input.manifest"
with open(manifest_name, "w") as f:
    for img in itertools.chain.from_iterable(ims.values()):
        img_path = "s3://{}/{}/images/{}.jpg".format(BUCKET, EXP_NAME, img)
        f.write('{"source-ref": "' + img_path + '"}\n')
s3.upload_file(manifest_name, BUCKET, EXP_NAME + "/" + manifest_name)

上記のセルを実行後、[S3 コンソール](https://console.aws.amazon.com/s3/) から `s3://BUCKET/EXP_NAME/images` に行くことができ、1000枚分の画像を確認できます。これらの画像を詳しく見ることをお勧めします！ AWS CLI を使うことで、ローカルのマシンにすべての画像をダウンロードすることができます。

## カテゴリを特定する

画像分類のラベリングジョブを実行するためには、アノテーターが選択するための分類を決める必要があります。この場合は、`["Musical Instrument", "Fruit", "Cheetah", "Tiger", "Snowman"]`のリストになります。ジョブの中では10個までクラスを指定することができます。これらのクラスは明確で具体的であることをお勧めします。カテゴリは相互に排他的で、各画像には１つの正しい画像が特定されるべきです。加えて、できる限りタスクを*客観的*にするように注意する必要があります。ただし、もちろん、主観的なラベルを得ることを意図している場合は、その限りではありません。
* よりカテゴリのリストの例: `["Human", "No Human"]`, `["Golden Retriever", "Labrador", "English Bulldog", "German Shepherd"]`, `["Car", "Train", "Ship", "Pedestrian"]`.
* 悪いカテゴリのリストの例: `["Prominent object", "Not prominent"]` (prominent=目立つ、の意味が明確でない), `["Beautiful", "Ugly"]` (主観的), `["Dog", "Animal", "Car"]` (相互に排他的でない). 

Ground Truth を使う場合、このリストは .json ファイルに変換され、S3 `BUCKET` にアップロードする必要があります。

*注：テンプレート内のラベルやクラスの順序は出力マニフェストで出てくるクラスのインデックスを規定するものになります（0から始まるインデックスです）。つまり、テンプレートで２番目に現れるクラスは、出力では "1" のクラスに対応します。このデモの終わりに、モデルの学習と推論を行いますが、このクラスの順序は結果を解釈するための手段となります。*

In [ ]:
CLASS_LIST = list(ims.keys())
print("Label space is {}".format(CLASS_LIST))

json_body = {"labels": [{"label": label} for label in CLASS_LIST]}
with open("class_labels.json", "w") as f:
    json.dump(json_body, f)

s3.upload_file("class_labels.json", BUCKET, EXP_NAME + "/class_labels.json")

実行すると、`s3://BUCKET/EXP_NAME/`に`class_labels.json`を確認できます。

## 指示書テンプレートの作成
一部またはすべての画像は人間のアノテーターによってアノテーションされます。したがって、アノテーターがあなたの欲しいようにアノテーションできるように良い指示書を提供することは**不可欠**です。良い指示書とは：
1. 簡潔。言語や本文での指示は２文に制限し、明確な画像に注力した方が良いです。
2. 画像。画像分類の場合は、指示書の一部として、各クラスごとのラベル付けされが画像を含めると良いです。

AWS コンソールをお使いの場合、Ground Truth ではビジュアルウィザードを使って作業書を作成することができます。API をお使いの場合、作業書のためには HTML テンプレートを作成する必要があります。下記では、とても簡単ですが有用なテンプレートを用意しており、あなたのS3 バケットにアップロードします。

注：（今回のように）画像をテンプレートで使用する場合、それらの画像はパブリックにアクセス可能である必要があります。S3 コンソールから S3 バケットないのファイルにパブリックなアクセスを許可することができます。詳細は、[S3 の開発者ガイド](https://docs.aws.amazon.com/AmazonS3/latest/user-guide/set-object-permissions.html)をご確認ください。

#### 指示書をテストする
破綻している指示書を作成するのは非常に簡単です。これによってラベリングジョブは失敗するかもしれません。もしかしたら、（アノテーターが何をすれば良いのかわからない時や、指示書が完全に間違っているときに）意味のない結果で完了することになるかもしれません。したがって、タスクが正しいかを次の２つの方法で確認することを*強く推奨します*：
1. 次のセルは、`instructions.template` というファイルを生成して、S3 にアップロードします。また、ローカルのブラウザで開くことのできる`instructions.html` というファイルを生成します。ぜひブラウザで開いてみて、生成された web ページを確認してください。あなたが（実際のアノテーションされる画像が表示されていないことを除いて）アノテーターに見えて欲しいようになっているべきです。
2. プライベートワークフォースを使用してジョブを実行してください。これは擬似的にラベリングジョブを実行する方法です。この方法については、[プライベートチームでタスクを検証する [オプション]](#プライベートチームでタスクを検証する-[オプション])の中で説明します。


In [ ]:
img_examples = [
    "https://s3.amazonaws.com/open-images-dataset/test/{}".format(img_id)
    for img_id in [
        "0634825fc1dcc96b.jpg",
        "0415b6a36f3381ed.jpg",
        "8582cc08068e2d0f.jpg",
        "8728e9fa662a8921.jpg",
        "926d31e8cde9055e.jpg",
    ]
]


def make_template(test_template=False, save_fname="instructions.template"):
    template = r"""<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
    <crowd-form>
      <crowd-image-classifier
        name="crowd-image-classifier"
        src="{{{{ task.input.taskObject | grant_read_access }}}}"
        header="Dear Annotator, please tell me what you can see in the image. Thank you!"
        categories="{categories_str}"
      >
        <full-instructions header="Image classification instructions">
        </full-instructions>

        <short-instructions>
          <p>Dear Annotator, please tell me whether what you can see in the image. Thank you!</p>
          <p><img src="{}" style="max-width:100%">
          <br>Example "Musical Instrument". </p>

          <p><img src="{}" style="max-width:100%">
          <br>Example "Fruit".</p>

          <p><img src="{}" style="max-width:100%">
          <br>Example "Cheetah". </p>

          <p><img src="{}" style="max-width:100%">
          <br>Example "Tiger". </p>

          <p><img src="{}" style="max-width:100%">
          <br>Example "Snowman". </p>

        </short-instructions>

      </crowd-image-classifier>
    </crowd-form>""".format(
        *img_examples,
        categories_str=str(CLASS_LIST)
        if test_template
        else "{{ task.input.labels | to_json | escape }}",
    )

    with open(save_fname, "w") as f:
        f.write(template)
    if test_template is False:
        print(template)


make_template(test_template=True, save_fname="instructions.html")
make_template(test_template=False, save_fname="instructions.template")
s3.upload_file("instructions.template", BUCKET, EXP_NAME + "/instructions.template")

テンプレートは `s3://BUCKET/EXP_NAME/instructions.template`で確認できます。

## タスクをテストするためのプライベートチームを作成する [オプション]
このステップでは AWS コンソールを使用します。しかし、特に独自のデータセットやラベルセット、テンプレートでタスクを作成するときに、このステップを行うことを**強く推奨します**。

ここでは`プライベートワークスチーム` を作り、ユーザーを一人（あなた）追加します。その後、Ground Truth のジョブリクエストの API を修正して、タスクをワークフォースに送信します。これにより、パブリックアノテーターが見る内容と同一のものをご自身で確認することができます。あるいは、全てのデータセットを自身でアノテーションすることもできます。

プライベートチームを作成する方法は、以下の通りです：
1. `AWS Console > Amazon SageMaker > Ground Truth > ラベリングワークフォース` に行きます。
1. `プライペート`をクリックし、"プライベートチーム"の下の`プライベートチームを作成`をクリックします。 
1. `チーム名`にプライベートチームの適当な名前を入力します。
1. `プライベートチームを作成`をクリックします。
1. 画面が `AWS Console > Amazon SageMaker > Ground Truth > ラベリングワークフォース` に戻ります。新しく作成したチームが、"プライベートチーム"の下に表示されます。`arn:aws:sagemaker:region-name-123456:workteam/private-crowd/team-name`のような`ARN`が表示されるので、それを下のセルにコピーします。
1. "ワーカー"の下の`新しいワーカーを招待`をクリックします。
1. `E メールアドレス`の欄に、自身のメールアドレスを入力します。
1. `新しいワーカーを招待`をクリックします。
1. `no-reply@verificationemail.com` からワークフォースのユーザー名、パスワードとログイン URL が入った E メールが送信されます。メール本文中の URL をクリックし、ユーザー名とパスワードでログインします (新しいパスワードの生成を求められます)。
1. `AWS Console > Amazon SageMaker > Ground Truth > ラベリングワークフォース` の画面で、"プライベートチーム"の下の作成したチーム名をクリックします。
1. `ワーカー`のタブを選択し、`チームにワーカーを追加`をクリックします。
1. 登録した E メールアドレスを選択し、`チームにワーカーを追加`をクリックします。

これで完了です！9 で表示したページが、あなたのプライベートワーカーのインターフェースです。下記の[プライベートチームでタスクを検証する [オプション]](#プライベートチームでタスクを検証する-[オプション]) の中でタスクを作成したとき、タスクはこの画面に現れます。"新しいワーカーを招待"ボタンをクリックすることで、同僚をラベリングジョブに招待することができます。

[SageMaker Ground Truth 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management-private.html) にはプライベートワークフォースの管理方法について、詳細なドキュメントがあります。

In [ ]:
private_workteam_arn = "<< YOUR PRIVATE WORKTEAM ARN >>"

## ラベリングジョブで使用する構築済み Lambda 関数を定義する
ジョブリクエストを送る前に、次の４つの主要な要素について ARN を定義する必要があります：1) ワークフォースのチーム、2)　アノテーション用統合 Lambda 関数、3) ラベリング前タスク用の Lambda 関数、4) 自動アノテーションを行う機械学習アルゴリズム。これらの関数は、リージョン名、AWS サービスアカウントナンバーの文字列で定義されており、サポートされているリージョンでノートブックを実行できるように下記でマッピングを定義します。

利用可能な ARN  については公式のドキュメントを参照してください：
* [SageMaker 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management-public.html)：ワークフォースチームの ARN の定義を記載しています。パブリックワークフォースを使用する場合、利用可能な選択肢は１つしかありません。プライベートチームを使用する場合、各チームの対応する ARN を確認してください。
* [SageMaker API リファレンス - HumanTaskConfig](https://docs.aws.amazon.com/sagemaker/latest/dg/API_HumanTaskConfig.html#SageMaker-Type-HumanTaskConfig-PreHumanTaskLambdaArn) ：他のワークフローで利用可能な、人間によるラベリングタスクの前に実行される Lambda 関数の ARN を記しています。
* [SageMaker API リファレンス - AnnotationConsolidationConfig](https://docs.aws.amazon.com/sagemaker/latest/dg/API_AnnotationConsolidationConfig.html#SageMaker-Type-AnnotationConsolidationConfig-AnnotationConsolidationLambdaArn)：他のワークフローで利用可能な、アノテーション用統合 Lambda 関数の ARN を記しています。
* [SageMaker API リファレンス - LabelingJobAlgorithmsConfig](https://docs.aws.amazon.com/sagemaker/latest/dg/API_LabelingJobAlgorithmsConfig.html#SageMaker-Type-LabelingJobAlgorithmsConfig-LabelingJobAlgorithmSpecificationArn)：他のワークフローで利用可能な、自動ラベリングアルゴリズムの ARN を記しています。

In [ ]:
# 画像分類ジョブの実行に必要なリソースの ARN を指定します.
ac_arn_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449",
    "ap-northeast-1": "477331159723",
}

prehuman_arn = "arn:aws:lambda:{}:{}:function:PRE-ImageMultiClass".format(
    region, ac_arn_map[region]
)
acs_arn = "arn:aws:lambda:{}:{}:function:ACS-ImageMultiClass".format(region, ac_arn_map[region])
labeling_algorithm_specification_arn = "arn:aws:sagemaker:{}:027400017018:labeling-job-algorithm-specification/image-classification".format(
    region
)
workteam_arn = "arn:aws:sagemaker:{}:394669845002:workteam/public-crowd/default".format(region)

## Ground Truth ジョブリクエストを送信する
下記の API でリクエストを送信することで、Ground Truth ジョブを開始できます。リクエストはアノテーションタスクの全ての設定を含み、AWS コンソールではデフォルト値に固定されていたジョブの細かな詳細も変更することができます。リクエストを生成するためのパラメータの詳細については、[SageMaker Ground Truth の API リファレンス](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateLabelingJob.html)に記載されています。

リクエストを送信後、AWS コンソールの`Amazon SageMaker > Ground Truth > ラベリングジョブ`でジョブを見ることができます。ジョブの進捗状況もここで確認できます。ジョブの完了には数時間かかります。ジョブが大きい場合（例えば、10万枚の画像がある場合）、速度と費用に対する自動ラベリングの効果が大きくなります。

### プライベートチームでタスクを検証する [オプション] 
[プライベートチームを作成する](#タスクをテストするためのプライベートチームを作成する-[オプション])の手順に従った場合、最初にタスクが期待通りかを確認することができます。
1. 下記のセルで `VERIFY_USING_PRIVATE_WORKFORCE` を `True` にする。
2. 下記の２つのセルを実行します。これでタスクが定義され、プライベートワークフォース（あなた）に送信されます。
3. 数分後、プライベートワークフォースのインターフェースでタスクを見ることができます（[プライベートチームを作成する](#タスクをテストするためのプライベートチームを作成する-[オプション])も参照）。タスクが期待通りに表示されることを確認してください。
4. 期待通りであれば、`VERIFY_USING_PRIVATE_WORKFORCE` を `False` にして、下記のセルを実際のタスクのために実行してください！

In [ ]:
VERIFY_USING_PRIVATE_WORKFORCE = True
USE_AUTO_LABELING = True

task_description = "What do you see: a {}?".format(" a ".join(CLASS_LIST))
task_keywords = ["image", "classification", "humans"]
task_title = task_description
job_name = "ground-truth-demo-" + str(int(time.time()))

human_task_config = {
    "AnnotationConsolidationConfig": {
        "AnnotationConsolidationLambdaArn": acs_arn,
    },
    "PreHumanTaskLambdaArn": prehuman_arn,
    "MaxConcurrentTaskCount": 200,  # 一度に 200 枚の画像がワークフォースのチームに送られる
    "NumberOfHumanWorkersPerDataObject": 1,  # 1 人の異なるワーカーが各画像のラベル付けに必要
    "TaskAvailabilityLifetimeInSeconds": 21600,  # すべてのタスクを 6 時間以内に完了しなければいけない
    "TaskDescription": task_description,
    "TaskKeywords": task_keywords,
    "TaskTimeLimitInSeconds": 300,  # 各画像を 5 分以内にラベル付けしなければいけない
    "TaskTitle": task_title,
    "UiConfig": {
        "UiTemplateS3Uri": "s3://{}/{}/instructions.template".format(BUCKET, EXP_NAME),
    },
}

if not VERIFY_USING_PRIVATE_WORKFORCE:
    human_task_config["PublicWorkforceTaskPrice"] = {
        "AmountInUsd": {
            "Dollars": 0,
            "Cents": 1,
            "TenthFractionsOfACent": 2,
        }
    }
    human_task_config["WorkteamArn"] = workteam_arn
else:
    human_task_config["WorkteamArn"] = private_workteam_arn

ground_truth_request = {
    "InputConfig": {
        "DataSource": {
            "S3DataSource": {
                "ManifestS3Uri": "s3://{}/{}/{}".format(BUCKET, EXP_NAME, manifest_name),
            }
        },
        "DataAttributes": {
            "ContentClassifiers": ["FreeOfPersonallyIdentifiableInformation", "FreeOfAdultContent"]
        },
    },
    "OutputConfig": {
        "S3OutputPath": "s3://{}/{}/output/".format(BUCKET, EXP_NAME),
    },
    "HumanTaskConfig": human_task_config,
    "LabelingJobName": job_name,
    "RoleArn": role,
    "LabelAttributeName": "category",
    "LabelCategoryConfigS3Uri": "s3://{}/{}/class_labels.json".format(BUCKET, EXP_NAME),
}

if USE_AUTO_LABELING and RUN_FULL_AL_DEMO:
    ground_truth_request["LabelingJobAlgorithmsConfig"] = {
        "LabelingJobAlgorithmSpecificationArn": labeling_algorithm_specification_arn
    }
sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_labeling_job(**ground_truth_request)

## ジョブの進捗状況を監視する
Ground Truth のジョブは完了までに数時間かかります（データセットが1万枚以上だと、さらに長くかかります！）。ジョブの進捗を確認する１つの方法は AWS コンソールを使用することです。このノートブックでは、Ground Truth の出力ファイルと Cloud Watch logs を使った監視を行います。次の２つのセルを使って、繰り返し確認することができます。

次のセルは繰り返し実行できます。`describe_labelging_job`リクエストを送ることで、ジョブが完了したかどうかを表示します。完了したら、'LabelingJobStatus'が 'Completed'と表示されます。

In [ ]:
sagemaker_client.describe_labeling_job(LabelingJobName=job_name)

次のセルは、ジョブに関するこれまでの状態をより詳細に抽出します。何回も繰り返すことができます。ここでは、下記の情報が表示されます：

* ラベリングジョブの各反復において、人間と機械それぞれがアノテー小んした画像の枚数を各カテゴリごとに表示します。
* Ground Truth により起動されたニューラルネットワークのトレーニングジョブの学習曲線 **(`RUN_FULL_AL_DEMO=True`で実行した場合のみ)**。
* 人間と機械によりアノテーションしたラベルの料金

料金体系を理解するためには、[料金ページ](https://aws.amazon.com/sagemaker/groundtruth/pricing/) を十分にご確認ください. 今回の場合、人間によるラベル付けは 1 回ごとに`$0.08 + 3 * $0.012 = $0.116` かかり、自動でのラベル付けは 1 回ごとに `$0.08`かかります。他にも、自動ラベリングの間、ニューラルネットの学習と推論に SageMaker インスタンスを使用する少額のコストがかかりますが、他のコストに比べればわずかなものです。

`RUN_FULL_AL_DEMO==True`とした場合、 ステップを複数回繰り返すことでジョブが実行されます。
* ステップ 1: Ground Truth は 10 枚の画像を 人間のアノテーターに「テスト用」として送信します。これらのアノテーションが成功すると、次のステップに進みます。
* ステップ 2: `MaxConcurrentTaskCount - 10` 枚 (今回の場合、190 枚) の画像を人間のアノテーターに送信し、アクティブ・ラーニングのトレーニングバッチを取得します。
* ステップ 3: 次の 200 枚の画像のバッチを人間のアノテーターに送信し、アクティブラーニングの検証用データセットを取得します。
* ステップ 4a: 自動ラベリングを実行するためのニューラルネットの学習を行います。自動で可能な限り多くのラベル付を行います。
* ステップ 4b: ラベル付けされなかったデータが有る場合、最大で 200 枚の画像を人間のアノテーターに送信します。
* すべてのデータがラベル付けされるまで、ステップ 4a と 4b を繰り返します。

`RUN_FULL_AL_DEMO==False`の場合、ステップ 1 と 2 のみ実行されます。

In [ ]:
from datetime import datetime
import glob
import shutil
HUMAN_PRICE = 0.116
AUTO_PRICE = 0.08

try:
    os.makedirs('ic_output_data/', exist_ok=False)
except FileExistsError:
    shutil.rmtree('ic_output_data/')
    
S3_OUTPUT = boto3.client('sagemaker').describe_labeling_job(LabelingJobName=job_name)[
    'OutputConfig']['S3OutputPath'] + job_name

# 人間によるアノテーションの結果をダウンロードする
!aws s3 cp {S3_OUTPUT + '/annotations/worker-response'} ic_output_data/worker-response --recursive --quiet
worker_times = []
worker_ids = []

# ここまでのすべてのアノテーションイベントの時間とワーカー ID を収集する
for annot_fname in glob.glob('ic_output_data/worker-response/**', recursive=True):
    if annot_fname.endswith('json'):
        with open(annot_fname, 'r') as f:
            annot_data = json.load(f)
        for answer in annot_data['answers']:
            annot_time = datetime.strptime(
                answer['submissionTime'], '%Y-%m-%dT%H:%M:%S.%fZ')
            annot_id = answer['workerId']
            worker_times.append(annot_time)
            worker_ids.append(annot_id)

sort_ids = np.argsort(worker_times)
worker_times = np.array(worker_times)[sort_ids]
worker_ids = np.array(worker_ids)[sort_ids]
cumulative_n_annots = np.cumsum([1 for _ in worker_times])

# 各一意のワーカー ID について、アノテーションの数を数える
annots_per_worker = np.zeros(worker_ids.size)
ids_store = set()
for worker_id_id, worker_id in enumerate(worker_ids):
    ids_store.add(worker_id)
    annots_per_worker[worker_id_id] = float(
        cumulative_n_annots[worker_id_id]) / len(ids_store)
    
# 各クラス、反復ごとに、人間によるアノテーションの数を数える
!aws s3 cp {S3_OUTPUT + '/annotations/consolidated-annotation/consolidation-response'} ic_output_data/consolidation-response --recursive --quiet
consolidated_classes = defaultdict(list)
consolidation_times = {}
consolidated_cost_times = []

for consolidated_fname in glob.glob('ic_output_data/consolidation-response/**', recursive=True):
    if consolidated_fname.endswith('json'):
        iter_id = int(consolidated_fname.split('/')[-2][-1])
        # 反復時間として、直近の統合イベントの時間を格納する
        iter_time = datetime.strptime(consolidated_fname.split('/')[-1], '%Y-%m-%d_%H:%M:%S.json')
        if iter_id in consolidation_times:
            consolidation_times[iter_id] = max(consolidation_times[iter_id], iter_time)
        else:
            consolidation_times[iter_id] = iter_time
        consolidated_cost_times.append(iter_time)
                                      
        with open(consolidated_fname, 'r') as f:
            consolidated_data = json.load(f)
        for consolidation in consolidated_data:
            consolidation_class = consolidation['consolidatedAnnotation']['content'][
                'category-metadata']['class-name']
            consolidated_classes[iter_id].append(consolidation_class)
total_human_labels = sum([len(annots) for annots in consolidated_classes.values()])
            
# 各クラス、反復ごとに、機会によるアノテーションの数を数える
!aws s3 cp {S3_OUTPUT + '/activelearning'} ic_output_data/activelearning --recursive --quiet
auto_classes = defaultdict(list)
auto_times = {}
auto_cost_times = []

for auto_fname in glob.glob('ic_output_data/activelearning/**', recursive=True):
    if auto_fname.endswith('auto_annotator_output.txt'):
        iter_id = int(auto_fname.split('/')[-3])
        with open(auto_fname, 'r') as f:
            annots = [' '.join(l.split()[1:]) for l in f.readlines()]
        for annot in annots:
            annot = json.loads(annot)
            time_str = annot['category-metadata']['creation-date']
            auto_time = datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f')
            auto_class = annot['category-metadata']['class-name']
            auto_classes[iter_id].append(auto_class)
            if iter_id in auto_times:
                auto_times[iter_id] = max(auto_times[iter_id], auto_time)
            else:
                auto_times[iter_id] = auto_time
            auto_cost_times.append(auto_time)
                
total_auto_labels = sum([len(annots) for annots in auto_classes.values()])
n_iters = max(len(auto_times), len(consolidation_times))

def get_training_job_data(training_job_name):
    logclient = boto3.client('logs')
    log_group_name = '/aws/sagemaker/TrainingJobs'
    log_stream_name = logclient.describe_log_streams(logGroupName=log_group_name,
        logStreamNamePrefix=training_job_name)['logStreams'][0]['logStreamName']
    train_log = logclient.get_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        startFromHead=True
    )
    events = train_log['events']
    next_token = train_log['nextForwardToken']
    while True:
        train_log = logclient.get_log_events(
            logGroupName=log_group_name,
            logStreamName=log_stream_name,
            startFromHead=True,
            nextToken=next_token
        )
        if train_log['nextForwardToken'] == next_token:
            break
        events = events + train_log['events']

    errors = []
    for event in events:
        msg = event['message']
        if 'Final configuration' in msg:
            num_samples = int(msg.split('num_training_samples\': u\'')[1].split('\'')[0])
        elif 'Validation-accuracy' in msg:
            errors.append(float(msg.split('Validation-accuracy=')[1]))

    errors = 1 - np.array(errors)
    return num_samples, errors

training_data = !aws s3 ls {S3_OUTPUT + '/training/'} --recursive
training_sizes = []
training_errors = []
training_iters = []
for line in training_data:
    if line.split('/')[-1] == 'model.tar.gz':
        training_job_name = line.split('/')[-3]
        n_samples, errors = get_training_job_data(training_job_name)
        training_sizes.append(n_samples)
        training_errors.append(errors)
        training_iters.append(int(line.split('/')[-5]))

plt.figure(facecolor='white', figsize=(14, 4), dpi=100)
ax = plt.subplot(131)
plt.title('Label counts ({} human, {} auto)'.format(
    total_human_labels, total_auto_labels))
cmap = plt.get_cmap('coolwarm')
for iter_id in consolidated_classes.keys():
    bottom = 0
    class_counter = Counter(consolidated_classes[iter_id])
    for cname_id, cname in enumerate(CLASS_LIST):
        if iter_id == 1:
            plt.bar(iter_id, class_counter[cname], width=.4, bottom=bottom,
                label=cname, color=cmap(cname_id / float(len(CLASS_LIST)-1)))
        else:
            plt.bar(iter_id, class_counter[cname], width=.4, bottom=bottom,
                color=cmap(cname_id / float(len(CLASS_LIST)-1)))

        bottom += class_counter[cname]
        
for iter_id in auto_classes.keys():
    bottom = 0
    class_counter = Counter(auto_classes[iter_id])
    for cname_id, cname in enumerate(CLASS_LIST):
        plt.bar(iter_id + .4, class_counter[cname], width=.4, bottom=bottom, color=cmap(cname_id / float(len(CLASS_LIST)-1)))
        bottom += class_counter[cname]

tick_labels_human = ['Iter {}, human'.format(iter_id + 1) for iter_id in range(n_iters)]
tick_labels_auto = ['Iter {}, auto'.format(iter_id + 1) for iter_id in range(n_iters)]
tick_locations_human = np.arange(n_iters) + 1
tick_locations_auto = tick_locations_human + .4
tick_labels = np.concatenate([[tick_labels_human[idx], tick_labels_auto[idx]] for idx in range(n_iters)])
tick_locations = np.concatenate([[tick_locations_human[idx], tick_locations_auto[idx]] for idx in range(n_iters)])
plt.xticks(tick_locations, tick_labels, rotation=90)
plt.legend()
plt.ylabel('Count')

ax = plt.subplot(132)
total_human = 0
total_auto = 0
for iter_id in range(1, n_iters + 1):
    cost_human = len(consolidated_classes[iter_id]) * HUMAN_PRICE
    cost_auto = len(auto_classes[iter_id]) * AUTO_PRICE
    total_human += cost_human
    total_auto += cost_auto
    
    plt.bar(iter_id, cost_human, width=.8, color='gray',
            hatch='/', edgecolor='k', label='human' if iter_id==1 else None)
    plt.bar(iter_id, cost_auto, bottom=cost_human,
            width=.8, color='gray', edgecolor='k', label='auto' if iter_id==1 else None)
plt.title('Annotation costs (\${:.2f} human, \${:.2f} auto)'.format(
    total_human, total_auto))
plt.xlabel('Iter')
plt.ylabel('Cost in dollars')
plt.legend()

if len(training_sizes) > 0:
    plt.subplot(133)
    plt.title('Active learning training curves')
    plt.grid(True)

    cmap = plt.get_cmap('coolwarm')
    n_all = len(training_sizes)
    for iter_id_id, (iter_id, size, errs) in enumerate(zip(training_iters, training_sizes, training_errors)):
        plt.plot(errs, label='Iter {}, auto'.format(iter_id + 1), color=cmap(iter_id_id / max(1, (n_all-1))))
        plt.legend()

    plt.xscale('log')
    plt.xlabel('Training epoch')
    plt.ylabel('Validation error')

# Ground Truth ラベリングジョブの結果を分析する
**このセクションは完了に 20 分程度かかります。**

ジョブの実行が終了すると（**`sagemaker_client.describe_labeling_job` でジョブの完了が表示されていることを確認してください！**）、結果を分析するときとなります。 [ジョブの進捗状況を監視する](#ジョブの進捗状況を監視する) セクションのグラフは分析の一部を成します。このセクションでは、`output.manifest` にすべて含まれた、結果に対するさらなるインサイトを得ます。output.manifest の場所は、`AWS Console > SageMaker > Ground Truth > ラベリングジョブ > [あなたのジョブ]` の「出力データセットの場所」配下の `manifests/output` から見つけることができます。 下記のセルでは、コードによって取得します。

## 出力マニフェストファイルの結果を後処理する
ジョブが完了したので、output.manifest をダウンロードして後処理を行い、次の配列を作ります：
* `img_uris` は Ground Truth でアノテーションされたすべての画像の S3 URI の配列です。
* `labels` は `img_uris` の画像の Ground Truth のラベルの配列です。
* `confidences` は `labels` の各ラベルの確信度の配列です。
* `human` はフラグの配列であり、各インデックスの画像において、1 は人間によるアノテーションであること、0 は Ground Truth の自動ラベリングでアノテーションされたことを示します。

In [ ]:
# 出力マニフェストのアノテーションの結果をダウンロードします
OUTPUT_MANIFEST = "s3://{}/{}/output/{}/manifests/output/output.manifest".format(
    BUCKET, EXP_NAME, job_name
)

!aws s3 cp {OUTPUT_MANIFEST} 'output.manifest'

with open("output.manifest", "r") as f:
    output = [json.loads(line.strip()) for line in f.readlines()]

# 上述の配列を作ります
img_uris = [None] * len(output)
confidences = np.zeros(len(output))
groundtruth_labels = [None] * len(output)
human = np.zeros(len(output))

# マニフェストに対応するジョブ名を取得します
keys = list(output[0].keys())
metakey = keys[np.where([("-metadata" in k) for k in keys])[0][0]]
jobname = metakey[:-9]

# データを抽出します
for datum_id, datum in enumerate(output):
    img_uris[datum_id] = datum["source-ref"]
    groundtruth_labels[datum_id] = str(datum[metakey]["class-name"])
    confidences[datum_id] = datum[metakey]["confidence"]
    human[datum_id] = int(datum[metakey]["human-annotated"] == "yes")
groundtruth_labels = np.array(groundtruth_labels)

## クラスに対するヒストグラムを作成する
それでは、クラスに対するヒストグラムを作成してみましょう。次のセルは、3 つの図を作成します：
* 左の図は、各視覚的なカテゴリに属するアノテーションされた画像の数をプロットします。これらのカテゴリは降順にソートされています。各バーは 'human' と 'machine' の部分に分けられ、それぞれ何枚の画像が人手もしくは自動ラベリングのメカニズムでアノテーションされたかを示しています。

* 真ん中の図は、左の図と同じですが、y 軸が対数スケールになっています。これにより、他と桁数が異なる画像を持ったカテゴリがあるような、偏ったデータセットを可視化することができます。

* 右の図は、各カテゴリの画像の平均の確信度を、人間と自動アノテーションそれぞれで表しています。

In [ ]:
# 各クラスのアノテーションの数を計算する
n_classes = len(set(groundtruth_labels))
sorted_clnames, class_sizes = zip(*Counter(groundtruth_labels).most_common(n_classes))

# 人によるアノテーションの数を数える
human_sizes = [human[groundtruth_labels == clname].sum() for clname in sorted_clnames]
class_sizes = np.array(class_sizes)
human_sizes = np.array(human_sizes)

# 各クラスの平均のアノテーションの確信度を計算する
human_confidences = np.array(
    [confidences[np.logical_and(groundtruth_labels == clname, human)] for clname in sorted_clnames]
)
machine_confidences = [
    confidences[np.logical_and(groundtruth_labels == clname, 1 - human)]
    for clname in sorted_clnames
]

# アノテーションされた画像が無いクラスには、0 の平均確信度をセットする
for class_id in range(n_classes):
    if human_confidences[class_id].size == 0:
        human_confidences[class_id] = np.array([0])
    if machine_confidences[class_id].size == 0:
        machine_confidences[class_id] = np.array([0])

plt.figure(figsize=(9, 3), facecolor="white", dpi=100)
plt.subplot(1, 3, 1)
plt.title("Annotation histogram")
plt.bar(range(n_classes), human_sizes, color="gray", hatch="/", edgecolor="k", label="human")
plt.bar(
    range(n_classes),
    class_sizes - human_sizes,
    bottom=human_sizes,
    color="gray",
    edgecolor="k",
    label="machine",
)
plt.xticks(range(n_classes), sorted_clnames, rotation=90)
plt.ylabel("Annotation Count")
plt.legend()

plt.subplot(1, 3, 2)
plt.title("Annotation histogram (logscale)")
plt.bar(range(n_classes), human_sizes, color="gray", hatch="/", edgecolor="k", label="human")
plt.bar(
    range(n_classes),
    class_sizes - human_sizes,
    bottom=human_sizes,
    color="gray",
    edgecolor="k",
    label="machine",
)
plt.xticks(range(n_classes), sorted_clnames, rotation=90)
plt.yscale("log")

plt.subplot(1, 3, 3)
plt.title("Mean confidences")
plt.bar(
    np.arange(n_classes),
    [conf.mean() for conf in human_confidences],
    color="gray",
    hatch="/",
    edgecolor="k",
    width=0.4,
)
plt.bar(
    np.arange(n_classes) + 0.4,
    [conf.mean() for conf in machine_confidences],
    color="gray",
    edgecolor="k",
    width=0.4,
)
plt.xticks(range(n_classes), sorted_clnames, rotation=90);

## アノテーションされた画像を表示する
どんなデータサイエンスのタスクにおいても、結果をプロットして検査し、意味が通っているかを確認することは非常に重要です。このために、ここでは、
1. Ground Truth がアノテーションした入力画像をダウンロードし、
2. カテゴリとアノテーションを人か自動ラベリングで行ったかによって分け、
3. それぞれの画像をプロットします。

入力画像は次のセルで指定した `LOCAL_IMAGE_DIR` にダウンロードします。もしこのディレクトリにすでに同じ名前の入力画像があれば、改めて画像をダウンロードすることはしませんので、注意してください。

もしデータセットが大きく、**すべての**画像をダウンロードして表示したくない場合は、単に `DATASET_SIZE` を小さい値にしてください。プロットするデータをランダムに抽出します。

In [ ]:
LOCAL_IMG_DIR = '<< choose a local directory name to download the images to >>' # 画像をダウンロードするローカルのディレクトリに置き換えてください
assert LOCAL_IMG_DIR != '<< choose a local directory name to download the images to >>', 'Please provide a local directory name'
DATASET_SIZE = len(img_uris) # データセットが1万枚以上の画像になる場合、適切な数に変更してください。

subset_ids = np.random.choice(range(len(img_uris)), DATASET_SIZE, replace=False)
img_uris = [img_uris[idx] for idx in subset_ids]
groundtruth_labels = groundtruth_labels[subset_ids]
confidences = confidences[subset_ids]
human = human[subset_ids]

img_fnames = [None] * len(output)
for img_uri_id, img_uri in enumerate(img_uris):
    target_fname = os.path.join(
        LOCAL_IMG_DIR, img_uri.split('/')[-1])
    if not os.path.isfile(target_fname):
        !aws s3 cp {img_uri} {target_fname}
    img_fnames[img_uri_id] = target_fname

### 数枚の出力サンプルを表示する
次のセルは 2 つの図を生成します。1 つ目では、各カテゴリの `N_SHOW` 枚の画像を、人によるアノテーションとして表示します。2 つ目は、`N_SHOW` 枚の画像を、自動ラベリングによるアノテーションとして表示します。

`N_SHOW` 枚以下の画像しかないカテゴリについては、その行はひょうじされません。デフォルトでは、を、`N_SHOW = 10`ですが、自由に異なる数に変更してください。

In [ ]:
N_SHOW = 10

plt.figure(figsize=(3 * N_SHOW, 2 + 3 * n_classes), facecolor="white", dpi=60)
for class_name_id, class_name in enumerate(sorted_clnames):
    class_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, human))[0]
    try:
        show_ids = class_ids[:N_SHOW]
    except ValueError:
        print("Not enough human annotations to show for class: {}".format(class_name))
        continue
    for show_id_id, show_id in enumerate(show_ids):
        plt.subplot2grid((n_classes, N_SHOW), (class_name_id, show_id_id))
        plt.title("Human Label: " + class_name)
        plt.imshow(imageio.imread(img_fnames[show_id]))  # image_fnames
        plt.axis("off")
    plt.tight_layout()

plt.figure(figsize=(3 * N_SHOW, 2 + 3 * n_classes), facecolor="white", dpi=100)
for class_name_id, class_name in enumerate(sorted_clnames):
    class_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, 1 - human))[0]
    try:
        show_ids = np.random.choice(class_ids, N_SHOW, replace=False)
    except ValueError:
        print("Not enough machine annotations to show for class: {}".format(class_name))
        continue
    for show_id_id, show_id in enumerate(show_ids):
        plt.subplot2grid((n_classes, N_SHOW), (class_name_id, show_id_id))
        plt.title("Auto Label: " + class_name)
        plt.imshow(imageio.imread(img_fnames[show_id]))
        plt.axis("off")
    plt.tight_layout()

### すべての結果を表示する
最後に、すべての結果を大きな PDF ファイルに表示します。（`ground_truth.pdf`の名前の）PDF は各ページに 100 枚の画像を表示します。各ページの画像はすべて同じカテゴリに属し、人手か自動ラベリングによるアノテーションのいずれかになります。この PDF を使うことで、正確にどの画像がどのクラスにアノテーションされたかをひと目で見る事ができます。

この処理は少し時間がかかり、生成された PDF はとても大きくなるかもしれません。1000 枚ほどの画像であれば、数分で 10MB ほどの PDF になります。各カテゴリのサンプルの数を制限したい場合、`N_SHOW_PER_CLASS` を小さい数にセットしてください。

In [ ]:
N_SHOW_PER_CLASS = np.inf
plt.figure(figsize=(10, 10), facecolor="white", dpi=100)

with PdfPages("ground_truth.pdf") as pdf:
    for class_name in sorted_clnames:
        # 人によって class_name とアノテーションされた画像をプロットする
        plt.clf()
        plt.text(0.1, 0.5, s="Images annotated as {} by humans".format(class_name), fontsize=20)
        plt.axis("off")

        class_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, human))[0]
        for img_id_id, img_id in enumerate(class_ids):
            if img_id_id == N_SHOW_PER_CLASS:
                break
            if img_id_id % 100 == 0:
                pdf.savefig()
                plt.clf()
                print(
                    "Plotting human annotations of {}, {}/{}...".format(
                        class_name, (img_id_id + 1), min(len(class_ids), N_SHOW_PER_CLASS)
                    )
                )
            plt.subplot(10, 10, (img_id_id % 100) + 1)
            plt.imshow(imageio.imread(img_fnames[img_id]), aspect="auto")
            plt.axis("off")
        pdf.savefig()

        # 機会によって class_name とアノテーションされた画像をプロットする
        plt.clf()
        plt.text(0.1, 0.5, s="Images annotated as {} by machines".format(class_name), fontsize=20)
        plt.axis("off")

        class_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, 1 - human))[
            0
        ]
        for img_id_id, img_id in enumerate(class_ids):
            if img_id_id == N_SHOW_PER_CLASS:
                break
            if img_id_id % 100 == 0:
                pdf.savefig()
                plt.clf()
                print(
                    "Plotting machine annotations of {}, {}/{}...".format(
                        class_name, (img_id_id + 1), min(len(class_ids), N_SHOW_PER_CLASS)
                    )
                )
            plt.subplot(10, 10, (img_id_id % 100) + 1)
            plt.imshow(imageio.imread(img_fnames[img_id]), aspect="auto")
            plt.axis("off")
        pdf.savefig()
plt.clf()

# Ground Truth の結果を、既知の事前にラベル付けされたデータと比較する
このセクションは完了に 5 分程度かかります。

時々（例えば、システムのベンチマークを行う場合など）、代替となるラベル付けされたデータセットがある場合があります。例えば、Open Images のデータにはプロのアノテーションワークフォースによって注意深くアノテーションされています。これらを使うことで、 Ground Truth のラベル付けした結果を事前のラベル付けされた結果と比較するさらなる分析を行うことが出来ます。この際、人手によってラベル付けされた画像は殆どの場合 100% の正確性が無いことを心に留めておいてください。そのため、ラベリングの正確性を「Ground Truth のラベルが（絶対的な意味で）どれほどよいか」と考えるのではなく、「ある標準またはラベルのセットにしっかりと沿っているか」と考える方が良いです。

## 精度を計算する
このセルでは、標準のラベルに対して、Ground Truth の精度を計算します。


[データの準備](#データの準備)では、 どの画像がどのカテゴリに属するかを示す `ims` ディレクトリを作成しました。それを `standard_labels[i]` が `i` 番目の画像のラベルを表すようなに `standard_labels` に変換し、対応する `groundtruth_labels[i]` が特定できるようにします。

これにより、混同行列をプロットして、Ground Truth によるラベルが標準のものにどれほどよく沿っているかを評価することができます。ここでは、全体のデータセットに対して混合行列をプロットし、人手によるアノテーションと自動アノテーションの行列を分けるようにします。

In [ ]:
def plot_confusion_matrix(
    cm, classes, title="Confusion matrix", normalize=False, cmap=plt.cm.Blues
):
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = "d" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j].astype(int), fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()


# 'ims' ディレクトリ（クラス名から画像のマッピング）を画像クラスのリストに変換する
standard_labels = []
for img_uri in img_uris:
    img_uri = img_uri.split("/")[-1].split(".")[0]
    standard_label = [cname for cname, imgs_in_cname in ims.items() if img_uri in imgs_in_cname][0]
    standard_labels.append(standard_label)
standard_labels = np.array(standard_labels)

# データセット全体の混合行列をプロットする
plt.figure(facecolor="white", figsize=(12, 4), dpi=100)
plt.subplot(131)
mean_err = 100 - np.mean(standard_labels == groundtruth_labels) * 100
cnf_matrix = confusion_matrix(standard_labels, groundtruth_labels)

np.set_printoptions(precision=2)
plot_confusion_matrix(
    cnf_matrix,
    classes=sorted(ims.keys()),
    title="Full annotation set error {:.2f}%".format(mean_err),
    normalize=False,
)

# 人によりアノテーションした Ground Truth のラベルの混合行列をプロットする
plt.subplot(132)
mean_err = 100 - np.mean(standard_labels[human == 1.0] == groundtruth_labels[human == 1.0]) * 100
cnf_matrix = confusion_matrix(standard_labels[human == 1.0], groundtruth_labels[human == 1.0])
np.set_printoptions(precision=2)
plot_confusion_matrix(
    cnf_matrix,
    classes=sorted(ims.keys()),
    title="Human annotation set (size {}) error {:.2f}%".format(int(sum(human)), mean_err),
    normalize=False,
)

# 自動でアノテーションした Ground Truth のラベルの混合行列をプロットする
if sum(human == 0.0) > 0:
    plt.subplot(133)
    mean_err = (
        100 - np.mean(standard_labels[human == 0.0] == groundtruth_labels[human == 0.0]) * 100
    )
    cnf_matrix = confusion_matrix(standard_labels[human == 0.0], groundtruth_labels[human == 0.0])
    np.set_printoptions(precision=2)
    plot_confusion_matrix(
        cnf_matrix,
        classes=sorted(ims.keys()),
        title="Auto-annotation set (size {}) error {:.2f}%".format(
            int(len(human) - sum(human)), mean_err
        ),
        normalize=False,
    )

## 正誤のアノテーションをプロットする

このセルは結果全体のプロットを繰り返します。ただし、推論結果を正しいほうから間違っている方にソートするので、すべての間違っている推論に対する標準のラベルを表示します。

In [ ]:
N_SHOW_PER_CLASS = np.inf
plt.figure(figsize=(10, 10), facecolor="white", dpi=100)

with PdfPages("ground_truth_benchmark.pdf") as pdf:
    for class_name in sorted_clnames:
        human_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, human))[0]
        auto_ids = np.where(np.logical_and(np.array(groundtruth_labels) == class_name, 1 - human))[
            0
        ]
        for class_ids_id, class_ids in enumerate([human_ids, auto_ids]):
            plt.clf()
            plt.text(
                0.1,
                0.5,
                s="Images annotated as {} by {}".format(
                    class_name, "humans" if class_ids_id == 0 else "machines"
                ),
                fontsize=20,
            )
            plt.axis("off")

            good_ids = class_ids[
                np.where(standard_labels[class_ids] == groundtruth_labels[class_ids])[0]
            ]
            bad_ids = class_ids[
                np.where(standard_labels[class_ids] != groundtruth_labels[class_ids])[0]
            ]
            for img_id_id, img_id in enumerate(np.concatenate([good_ids, bad_ids])):
                if img_id_id == N_SHOW_PER_CLASS:
                    break
                if img_id_id % 100 == 0:
                    pdf.savefig()
                    plt.clf()
                    print(
                        "Plotting annotations of {}, {}/{}...".format(
                            class_name, img_id_id, min(len(class_ids), N_SHOW_PER_CLASS)
                        )
                    )
                ax = plt.subplot(10, 10, (img_id_id % 100) + 1)
                plt.imshow(imageio.imread(img_fnames[img_id]), aspect="auto")
                plt.axis("off")
                if img_id_id < len(good_ids):
                    # Draw a green border around the image.
                    rec = matplotlib.patches.Rectangle(
                        (0, 0), 1, 1, lw=10, edgecolor="green", fill=False, transform=ax.transAxes
                    )
                else:
                    # Draw a red border around the image.
                    rec = matplotlib.patches.Rectangle(
                        (0, 0), 1, 1, lw=10, edgecolor="red", fill=False, transform=ax.transAxes
                    )
                ax.add_patch(rec)
            pdf.savefig()
plt.clf()

# Ground Truth を使った画像分類器のトレーニング
ここまでで私達は完全なラベル付けされたデータセットを作成し、ここから最初のほうで定義したカテゴリに基づく画像を分類する機械学習モデルのトレーニングを行うことができます。ラベリングジョブの**拡張マニフェスト** を使うことで、さらなる変換や処理を行う必要なく、このトレーニングを実行できます。拡張マニフェストに関する完全な説明は、別の[サンプルノートブック](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/ground_truth_labeling_jobs/object_detection_augmented_manifest_training/object_detection_augmented_manifest_training.ipynb)を参照してください。

**注：** 高い精度を持つニューラルネットの学習には、ハイパーパラメータの選択を入念に行う必要があることが多いです。今回の場合、データセットに十分に良いハイパーパラメータを人手で指定しています。今回のニューラルネットは**100 データに対しておおよそ 60%、1000 データに対して 95% 以上の**精度を持つことができるはずです。 真新しいデータに対してニューラルネットワークの学習を行うには、[SageMaker モデルのハイパーパラメータチューニング](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html)を利用することを検討してください。

最初に、拡張マニフェストを学習用と検証用に 80/20 の割合で分割します。

In [ ]:
with open("output.manifest", "r") as f:
    output = [json.loads(line) for line in f.readlines()]

# 出力結果の配列内をシャッフルします
np.random.shuffle(output)

dataset_size = len(output)
train_test_split_index = round(dataset_size * 0.8)

train_data = output[:train_test_split_index]
validation_data = output[train_test_split_index:]

num_training_samples = 0
with open("train.manifest", "w") as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write("\n")
        num_training_samples += 1

with open("validation.manifest", "w") as f:
    for line in validation_data:
        f.write(json.dumps(line))
        f.write("\n")

次に、これらのマニフェストファイルを冒頭で定義した S3 バケットにアップロードし、トレーニングジョブで使用できるようにします。

In [ ]:
s3.upload_file("train.manifest", BUCKET, EXP_NAME + "/train.manifest")
s3.upload_file("validation.manifest", BUCKET, EXP_NAME + "/validation.manifest")

In [ ]:
# 一意なジョブ名を作成します
nn_job_name_prefix = "groundtruth-augmented-manifest-demo"
timestamp = time.strftime("-%Y-%m-%d-%H-%M-%S", time.gmtime())
nn_job_name = nn_job_name_prefix + timestamp

training_image = sagemaker.image_uris.retrieve(
    "image-classification", boto3.Session().region_name
)

training_params = {
    "AlgorithmSpecification": {"TrainingImage": training_image, "TrainingInputMode": "Pipe"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/output/".format(BUCKET, EXP_NAME)},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.p3.2xlarge", "VolumeSizeInGB": 50},
    "TrainingJobName": nn_job_name,
    "HyperParameters": {
        "epochs": "30",
        "image_shape": "3,224,224",
        "learning_rate": "0.01",
        "lr_scheduler_step": "10,20",
        "mini_batch_size": "16",
        "num_classes": str(num_classes),
        "num_layers": "18",
        "num_training_samples": str(num_training_samples),
        "resize": "224",
        "use_pretrained_model": "1",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 86400},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile",
                    "S3Uri": "s3://{}/{}/{}".format(BUCKET, EXP_NAME, "train.manifest"),
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": ["source-ref", "category"],
                }
            },
            "ContentType": "application/x-recordio",
            "RecordWrapperType": "RecordIO",
            "CompressionType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile",
                    "S3Uri": "s3://{}/{}/{}".format(BUCKET, EXP_NAME, "validation.manifest"),
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": ["source-ref", "category"],
                }
            },
            "ContentType": "application/x-recordio",
            "RecordWrapperType": "RecordIO",
            "CompressionType": "None",
        },
    ],
}

それでは、SageMaker のトレーニングジョブを生成します。

In [ ]:
sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_training_job(**training_params)

# トレーニングジョブが開始したことを確認します
print("Transform job started")
while True:
    status = sagemaker_client.describe_training_job(TrainingJobName=nn_job_name)[
        "TrainingJobStatus"
    ]
    if status == "Completed":
        print("Transform job ended with status: " + status)
        break
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")
    time.sleep(30)

# モデルをデプロイする

ここまでで私達は完全なラベル付けされたデータセットを作成し、モデルの学習をしたので、このモデルを使って推論を実行しましょう。

現在の画像分類は、.jpg と .png 形式の画像のみを推論のインプットとしてサポートします。出力はすべてのクラスに対する確率値を JSON 形式、もしくはバッチ形式の場合は JSON Lines 形式で表したものになります。

このセクションは複数のステップから成っています：

    モデルの生成 - トレーニングアウトプットに対してモデルを生成します
    バッチ変換 - バッチ推論のための変換ジョブを生成します
    リアルタイム推論のためのモデルのホスト - 推論エンドポイントを生成し、リアルタイム推論を実施します

## モデルの生成

In [ ]:
timestamp = time.strftime("-%Y-%m-%d-%H-%M-%S", time.gmtime())
model_name = "groundtruth-demo-ic-model" + timestamp
print(model_name)
info = sagemaker_client.describe_training_job(TrainingJobName=nn_job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]
print(model_data)

primary_container = {
    "Image": training_image,
    "ModelDataUrl": model_data,
}

create_model_response = sagemaker_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

## バッチ変換
それでは、上で作成したモデルを使ってバッチ推論を行うための SageMaker バッチ変換ジョブを作成します。

### テストデータをダウンロードする
まず、学習および検証データで試用しなかったテスト画像をダウンロードします。

In [ ]:
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "image-classification-model" + timestamp
batch_input = 's3://{}/{}/test/'.format(BUCKET, EXP_NAME)
batch_output = 's3://{}/{}/{}/output/'.format(BUCKET, EXP_NAME, batch_job_name)

# 各クラスから、ニューラルネットにとって新しい 2 枚の画像を、ローカルのバケットにコピーする
test_images = []
for class_id in ['/m/04szw', '/m/02xwb', '/m/0cd4d', '/m/07dm6', '/m/0152hh']:
    test_images.extend([label[0] + '.jpg' for label in all_labels if (label[2] == class_id and label[3] == '1')][-2:])
    
!aws s3 rm $batch_input --recursive
for test_img in test_images:
    !aws s3 cp s3://open-images-dataset/test/{test_img} {batch_input}

In [ ]:
request = {
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms": 16,
    "MaxPayloadInMB": 6,
    "BatchStrategy": "SingleRecord",
    "TransformOutput": {
        "S3OutputPath": "s3://{}/{}/{}/output/".format(BUCKET, EXP_NAME, batch_job_name)
    },
    "TransformInput": {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": batch_input}},
        "ContentType": "application/x-image",
        "SplitType": "None",
        "CompressionType": "None",
    },
    "TransformResources": {"InstanceType": "ml.p2.xlarge", "InstanceCount": 1},
}

print("Transform job name: {}".format(batch_job_name))

In [ ]:
sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while True:
    response = sagemaker_client.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    if status == "Completed":
        print("Transform job ended with status: " + status)
        break
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")
    time.sleep(30)

ジョブが完了したあと、推論結果を確認します。

In [ ]:
CLASS_LIST = ["Musical Instrument", "Fruit", "Cheetah", "Tiger", "Snowman"]
def get_label(out_fname):
    !aws s3 cp {out_fname} .
    print(out_fname)
    with open(out_fname.split('/')[-1]) as f:
        data = json.load(f)
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    print("Result: label - " + CLASS_LIST[index] + ", probability - " + str(probability))
    input_fname = out_fname.split('/')[-1][:-4]
    return CLASS_LIST[index], probability, input_fname

# Show prediction results.
!rm test_inputs/*
plt.figure(facecolor='white', figsize=(7, 15), dpi=100)
outputs = !aws s3 ls {batch_output}
outputs = [get_label(batch_output + prefix.split()[-1]) for prefix in outputs]
outputs.sort(key=lambda pred: pred[1], reverse=True)

for fname_id, (pred_cname, pred_conf, pred_fname) in enumerate(outputs):
    !aws s3 cp {batch_input}{pred_fname} test_inputs/{pred_fname}
    plt.subplot(5, 2, fname_id+1) 
    img = imageio.imread('test_inputs/{}'.format(pred_fname))
    plt.imshow(img)
    plt.axis('off')
    plt.title('{}\nconfidence={:.2f}'.format(pred_cname, pred_conf))
    
if RUN_FULL_AL_DEMO:
    warning = ''
else:
    warning = ('\nNOTE: In this small demo we only used 80 images to train the neural network.\n'
               'The predictions will be far from perfect! Set RUN_FULL_AL_DEMO=True to see properly trained results.')
plt.suptitle('Predictions sorted by confidence.{}'.format(warning))

## リアルタイム推論

それでは、モデルをホストしてエンドポイントを作成し、リアルタイム推論を行います。

このセクションは複数のステップから成っています：

    エンドポイント設定を作成する - エンドポイントを定義する設定を作成します
    エンドポイントを作成する - 設定を試用して推論エンドポイントを作成します
    推論を実行する - エンドポイントを利用して、入力したデータに対して推論を実行します
    クリーンアップ - エンドポイントとモデルを削除します

### エンドポイント設定を作成する

In [ ]:
timestamp = time.strftime("-%Y-%m-%d-%H-%M-%S", time.gmtime())
endpoint_config_name = job_name + "-epc" + timestamp
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint configuration name: {}".format(endpoint_config_name))
print("Endpoint configuration arn:  {}".format(endpoint_config_response["EndpointConfigArn"]))

### エンドポイントを作成する

最後に、名前と先程定義した設定を使って、モデルを提供するためのエンドポイントを作成します。最終的には、エンドポイントは検証され、商用のアプリケーションに組み込まれます。この実行には、10分ほどかかります。

In [ ]:
timestamp = time.strftime("-%Y-%m-%d-%H-%M-%S", time.gmtime())
endpoint_name = job_name + "-ep" + timestamp
print("Endpoint name: {}".format(endpoint_name))

endpoint_params = {
    "EndpointName": endpoint_name,
    "EndpointConfigName": endpoint_config_name,
}
endpoint_response = sagemaker_client.create_endpoint(**endpoint_params)
print("EndpointArn = {}".format(endpoint_response["EndpointArn"]))

# エンドポイントの状態を取得する
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = response["EndpointStatus"]
print("EndpointStatus = {}".format(status))

# 状態が変わるまで待つ
sagemaker_client.get_waiter("endpoint_in_service").wait(EndpointName=endpoint_name)

# エンドポイントの状態を出力する
endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response["EndpointStatus"]
print("Endpoint creation ended with EndpointStatus = {}".format(status))

if status != "InService":
    raise Exception("Endpoint creation failed.")

### 推論を実行する

In [ ]:
with open("test_inputs/{}".format(test_images[0]), "rb") as f:
    payload = f.read()
    payload = bytearray(payload)

client = boto3.client("sagemaker-runtime")
response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/x-image", Body=payload
)

# `response` は JSON 形式で取得されるため、展開する
result = json.loads(response["Body"].read())
# 出力された結果は、全てのクラスの確率値
# 最大の確率を持つクラスを見つけ、出力する
print("Model prediction is: {}".format(CLASS_LIST[np.argmax(result)]))

最後に、エンドポイントを削除して、クリーンアップします。

In [ ]:
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# まとめ

このノートブックでは、様々な概念についれ触れていきました！ここまで達成してきたことを振り返ってみましょう。まず、私達はラベルの無いデータセット（技術的には、データセットの作者によって事前にラベル付けされていましたが、このデモのために元のラベルは一旦破棄しています）。次に、SageMake Ground Truth のラベリングジョブを作成し、データセット内のすべての画像に対してラベルを付与しました。その後、このファイルを学習用と検証用に分割し、SageMaker の画像分類モデルをトレーニングしました。最後に、モデルのエンドポイントをホストし、元のデータセットで残しておいた画像に対して、エンドポイントを使ったリアルタイム推論を行いました。
